In [2]:
from env_loader import load_env

repositoryDir= "./repository"
docs= [ ]

# Load OpenAI and ActiveLoop tokens into environment
load_env("./")

In [3]:
import os
from langchain.document_loaders import TextLoader

# Load all files inside the repository
for dirpath, dirnames, filenames in os.walk(repositoryDir):
  for file in filenames:
    try:
      loader= TextLoader(os.path.join(dirpath, file), encoding= "utf-8")
      docs.extend(loader.load_and_split( ))
    except Exception as e:
      pass

In [4]:
from langchain.text_splitter import CharacterTextSplitter

# Chunk the files
texts = CharacterTextSplitter(chunk_size= 1000, chunk_overlap= 0).split_documents(docs)

Created a chunk of size 1020, which is longer than the specified 1000
Created a chunk of size 1540, which is longer than the specified 1000
Created a chunk of size 1083, which is longer than the specified 1000
Created a chunk of size 1676, which is longer than the specified 1000
Created a chunk of size 1162, which is longer than the specified 1000


In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

try:
  # Create DeepLake dataset
  global db
  db= DeepLake(
    dataset_path= "hub://archis/nicobraun-kobold",
    embedding_function= OpenAIEmbeddings(disallowed_special=( )),
    public= True
  )
except:
  db= DeepLake(
    dataset_path= "hub://archis/nicobraun-kobold",
    embedding_function= OpenAIEmbeddings(disallowed_special=( )),
  )

/home/archi/Documents/langchain-demo/venv/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.5) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/archis/nicobraun-kobold



-

hub://archis/nicobraun-kobold loaded successfully.

Deep Lake Dataset in hub://archis/nicobraun-kobold already exists, loading from the storage
Dataset(path='hub://archis/nicobraun-kobold', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (289, 1536)  float32   None   
    ids      text     (289, 1)      str     None   
 metadata    json     (289, 1)      str     None   
   text      text     (289, 1)      str     None   


In [6]:
# Create embeddings and upload to ActiveLoop
# db.add_documents(texts)

Evaluating ingest: 100%|██████████| 1/1 [00:10<00:00
/

Dataset(path='hub://archis/nicobraun-kobold', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (289, 1536)  float32   None   
    ids      text     (289, 1)      str     None   
 metadata    json     (289, 1)      str     None   
   text      text     (289, 1)      str     None   


['37892cbe-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cbf-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cc0-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cc1-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cc2-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cc3-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cc4-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cc5-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cc6-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cc7-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cc8-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cc9-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cca-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892ccb-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892ccc-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892ccd-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cce-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892ccf-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cd0-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cd1-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cd2-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cd3-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cd4-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cd5-0fa5-11ee-a0ae-ff609d40bfd5',
 '37892cd6-0fa5-

In [7]:
# Construct the retriever, then construct the Conversational Chain
retriever = db.as_retriever( )
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 10

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name= "gpt-3.5-turbo")
qa = ConversationalRetrievalChain.from_llm(model, retriever= retriever, chain_type= "refine")

In [14]:
chatHistory= [ ]

def chat(questions):
  for question in questions:
    result = qa({ "question": question, "chat_history": chatHistory })

    chatHistory.append((question, result["answer"]))

    print(f"=> Question |: {question} \n")
    print(f"Answer |: {result['answer']} \n")

In [15]:
chat([
  """
    Explain the contents in file /internal/server/server.go
  """
])

=> Question |: 
    Explain the contents in file /internal/server/server.go
   

Answer |: The additional context provided in the form of imported packages from the `go.mod` file does not provide any relevant information about the contents of the `/internal/server/server.go` file. The original answer still stands that the provided code snippet appears to be setting up and running a web server using the `server` package imported from the `internal` directory, and the `flag`, `filepath`, `os`, and `panic` packages are being used for parsing command line arguments and handling errors. 

